In [11]:
import numpy as np

### Random matrix generation

We need matrix of our quadratic function to be positive definite. The definition is: 

$$ M \text { positive definite } \Longleftrightarrow x^{\top} M x>0 \text { for all } x \in \mathbb{R}^{n} \backslash \mathbf{0} $$

And we know that matrix is p.d. if and only if all of it's eigenvalues are positive. So i have a little function that check if matrix is p.d. Actually we don't really need this, becouse if we create our matrix as $A=A*A^T$ we will definitely get symmetric and p.d. matrix (1-st is obvious, 2-nd is becouse <a href="https://en.wikipedia.org/wiki/Cholesky_decomposition">Cholesky decomposition</a>)

In [12]:
def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)

In [13]:
n = 3 # size 

A = np.random.rand(n, n)
A = A@A.T

while not is_pos_def(A):
    A = np.random.rand(n, n)
    A = A@A.T
b = np.random.rand(n, 1)

eps = 1e-6 # precision 

In [14]:
print(A)
print(b)

[[0.85289924 0.76734277 0.81831045]
 [0.76734277 0.9050484  0.7206928 ]
 [0.81831045 0.7206928  1.14945543]]
[[0.01281577]
 [0.86473599]
 [0.65472843]]


$$f(x)=\frac{1}{2} x^{T} A x+x^{T} b$$

We need result to be scalar, but numpy will give as array. So we use np.asscalar

In [15]:
def f(x):
    return np.asscalar(1/2*x.T@A@x + b.T@x)

### Steepest gradient descent

The theory behind this is simple. Let imagine that we have $x^k$ and we want to build $x^{k+1}$ that $f(x^{k+1}) < f(x^k)$. Let $x^{k+1} = x^k+\mu q$. So $q$ is direction, $\mu$ is step size. We can use condition on global minimum to find $\mu$. For our simple and convinient function it sounds like: $\dot{f}(x)=0$. So 
$$\begin{aligned} f\left(x^{k+1}\right) &=f\left(x^{k}+\mu q\right) \equiv \varphi(\mu)=\varphi(0)+\mu \dot{\varphi}(0)+\frac{1}{2} \mu^{2} \ddot{\varphi}(0), \quad \dot{\varphi}=\frac{d \varphi}{d \mu} \end{aligned}$$
Where 
$$\varphi(0)=f\left(x^{k}\right), \dot{\varphi}(0)=q^{T}\left(A x^{k}+b\right), \ddot{\varphi}(0)=q^{T} A q$$
And finally 
$$\bar{\mu}_{k}=-\frac{\dot{\varphi}(0)}{\ddot{\varphi}(0)}=-\frac{q^{T}\left(A x^{k}+b\right)}{q^{T} \cdot A q}$$

We build our ${x^{k+1}}$ using this formulas and get decreasing sequence for $f(x)$. And eventually we get our extrema 

The only thing left is direction of descent. And this is the only part that differ Gradient descent and Coordinate descent. 

In sgd we can take gradient of our function ($\operatorname{grad} f(x)=\left(\frac{\partial f(x)}{\partial x_{1}}, \frac{\partial f(x)}{\partial x_{2}}, \dots \frac{\partial f(x)}{\partial x_{n}}\right)^{T}$), in our case it would be $q=\operatorname{grad} f(x)=A x+b$

In cd we can take $q=e^{i}=(\underbrace{0,0, \ldots, 0,1}_{i}, 0 \ldots, 0)^{T}$ where $e^i$ - i-th unit vector in our vector space

In [16]:
def sgd():
    x = b # init approximation 
    i = 0 # count 

    while True:
        q = A@x + b # grad 

        mu = q.T @ q / (q.T @ A @ q)

        x_old = x
        x = x - np.asscalar(mu) * q

        i += 1
        if((np.linalg.norm(x - x_old) < eps and np.linalg.norm(q) < eps) or i > 1000000):
            break
    return x, i

In [17]:
x_sd, i_sd = sgd()

In [18]:
x_sd, i_sd

(array([[ 5.54689878],
        [-4.11457399],
        [-1.93871786]]), 57)

In [19]:
f(x_sd)

-2.3781336140781653

### Coordinate descent

E - is utility matrix that allow us to extract unit vectors from it

In [20]:
E = np.eye(n) 
E = np.matrix(E)

In [43]:
def cd():
    x = b
    i = 0

    while True: 
        for j in range(n):
            q = E[j].T
            mu = q.T @ (A[j]+ b)[j] / A[j][j]
            
            x_old = x
            x = x - np.asscalar(mu) * q
            print(x)
        i += 1
        if((np.linalg.norm(x - x_old) < eps and np.linalg.norm(A@x+b) < eps) or i > 1000000):
            break
    return x, i

In [44]:
x_coord, i_coord = cd()

[[-1.00221035]
 [ 0.86473599]
 [ 0.65472843]]
[[-1.00221035]
 [-1.0907223 ]
 [ 0.65472843]]
[[-1.00221035]
 [-1.0907223 ]
 [-0.91487037]]
[[-2.01723648]
 [-1.0907223 ]
 [-0.91487037]]
[[-2.01723648]
 [-3.04618058]
 [-0.91487037]]
[[-2.01723648]
 [-3.04618058]
 [-2.48446917]]
[[-3.03226261]
 [-3.04618058]
 [-2.48446917]]
[[-3.03226261]
 [-5.00163887]
 [-2.48446917]]
[[-3.03226261]
 [-5.00163887]
 [-4.05406797]]
[[-4.04728874]
 [-5.00163887]
 [-4.05406797]]
[[-4.04728874]
 [-6.95709716]
 [-4.05406797]]
[[-4.04728874]
 [-6.95709716]
 [-5.62366677]]
[[-5.06231487]
 [-6.95709716]
 [-5.62366677]]
[[-5.06231487]
 [-8.91255545]
 [-5.62366677]]
[[-5.06231487]
 [-8.91255545]
 [-7.19326556]]
[[-6.07734099]
 [-8.91255545]
 [-7.19326556]]
[[ -6.07734099]
 [-10.86801374]
 [ -7.19326556]]
[[ -6.07734099]
 [-10.86801374]
 [ -8.76286436]]
[[ -7.09236712]
 [-10.86801374]
 [ -8.76286436]]
[[ -7.09236712]
 [-12.82347203]
 [ -8.76286436]]
[[ -7.09236712]
 [-12.82347203]
 [-10.33246316]]
[[ -8.10739325]
 [-

[[-254.75874235]
 [-487.99983604]
 [-391.74497141]]
[[-254.75874235]
 [-489.95529433]
 [-391.74497141]]
[[-254.75874235]
 [-489.95529433]
 [-393.31457021]]
[[-255.77376848]
 [-489.95529433]
 [-393.31457021]]
[[-255.77376848]
 [-491.91075262]
 [-393.31457021]]
[[-255.77376848]
 [-491.91075262]
 [-394.88416901]]
[[-256.78879461]
 [-491.91075262]
 [-394.88416901]]
[[-256.78879461]
 [-493.86621091]
 [-394.88416901]]
[[-256.78879461]
 [-493.86621091]
 [-396.45376781]]
[[-257.80382073]
 [-493.86621091]
 [-396.45376781]]
[[-257.80382073]
 [-495.82166919]
 [-396.45376781]]
[[-257.80382073]
 [-495.82166919]
 [-398.02336661]]
[[-258.81884686]
 [-495.82166919]
 [-398.02336661]]
[[-258.81884686]
 [-497.77712748]
 [-398.02336661]]
[[-258.81884686]
 [-497.77712748]
 [-399.59296541]]
[[-259.83387299]
 [-497.77712748]
 [-399.59296541]]
[[-259.83387299]
 [-499.73258577]
 [-399.59296541]]
[[-259.83387299]
 [-499.73258577]
 [-401.16256421]]
[[-260.84889912]
 [-499.73258577]
 [-401.16256421]]
[[-260.84889

[[-455.73391569]
 [-877.13603538]
 [-704.09513249]]
[[-456.74894182]
 [-877.13603538]
 [-704.09513249]]
[[-456.74894182]
 [-879.09149367]
 [-704.09513249]]
[[-456.74894182]
 [-879.09149367]
 [-705.66473129]]
[[-457.76396795]
 [-879.09149367]
 [-705.66473129]]
[[-457.76396795]
 [-881.04695196]
 [-705.66473129]]
[[-457.76396795]
 [-881.04695196]
 [-707.23433009]]
[[-458.77899408]
 [-881.04695196]
 [-707.23433009]]
[[-458.77899408]
 [-883.00241024]
 [-707.23433009]]
[[-458.77899408]
 [-883.00241024]
 [-708.80392889]]
[[-459.7940202 ]
 [-883.00241024]
 [-708.80392889]]
[[-459.7940202 ]
 [-884.95786853]
 [-708.80392889]]
[[-459.7940202 ]
 [-884.95786853]
 [-710.37352769]]
[[-460.80904633]
 [-884.95786853]
 [-710.37352769]]
[[-460.80904633]
 [-886.91332682]
 [-710.37352769]]
[[-460.80904633]
 [-886.91332682]
 [-711.94312649]]
[[-461.82407246]
 [-886.91332682]
 [-711.94312649]]
[[-461.82407246]
 [-888.86878511]
 [-711.94312649]]
[[-461.82407246]
 [-888.86878511]
 [-713.51272529]]
[[-462.83909

 [ -892.44698841]]
[[ -578.55207718]
 [-1111.79102996]
 [ -892.44698841]]
[[ -578.55207718]
 [-1113.74648824]
 [ -892.44698841]]
[[ -578.55207718]
 [-1113.74648824]
 [ -894.01658721]]
[[ -579.56710331]
 [-1113.74648824]
 [ -894.01658721]]
[[ -579.56710331]
 [-1115.70194653]
 [ -894.01658721]]
[[ -579.56710331]
 [-1115.70194653]
 [ -895.58618601]]
[[ -580.58212943]
 [-1115.70194653]
 [ -895.58618601]]
[[ -580.58212943]
 [-1117.65740482]
 [ -895.58618601]]
[[ -580.58212943]
 [-1117.65740482]
 [ -897.15578481]]
[[ -581.59715556]
 [-1117.65740482]
 [ -897.15578481]]
[[ -581.59715556]
 [-1119.61286311]
 [ -897.15578481]]
[[ -581.59715556]
 [-1119.61286311]
 [ -898.72538361]]
[[ -582.61218169]
 [-1119.61286311]
 [ -898.72538361]]
[[ -582.61218169]
 [-1121.5683214 ]
 [ -898.72538361]]
[[ -582.61218169]
 [-1121.5683214 ]
 [ -900.29498241]]
[[ -583.62720782]
 [-1121.5683214 ]
 [ -900.29498241]]
[[ -583.62720782]
 [-1123.52377969]
 [ -900.29498241]]
[[ -583.62720782]
 [-1123.52377969]
 [ -901.86

 [-1171.8355747 ]]
[[ -759.22672796]
 [-1461.81806353]
 [-1173.4051735 ]]
[[ -760.24175409]
 [-1461.81806353]
 [-1173.4051735 ]]
[[ -760.24175409]
 [-1463.77352182]
 [-1173.4051735 ]]
[[ -760.24175409]
 [-1463.77352182]
 [-1174.9747723 ]]
[[ -761.25678021]
 [-1463.77352182]
 [-1174.9747723 ]]
[[ -761.25678021]
 [-1465.72898011]
 [-1174.9747723 ]]
[[ -761.25678021]
 [-1465.72898011]
 [-1176.5443711 ]]
[[ -762.27180634]
 [-1465.72898011]
 [-1176.5443711 ]]
[[ -762.27180634]
 [-1467.6844384 ]
 [-1176.5443711 ]]
[[ -762.27180634]
 [-1467.6844384 ]
 [-1178.1139699 ]]
[[ -763.28683247]
 [-1467.6844384 ]
 [-1178.1139699 ]]
[[ -763.28683247]
 [-1469.63989668]
 [-1178.1139699 ]]
[[ -763.28683247]
 [-1469.63989668]
 [-1179.6835687 ]]
[[ -764.3018586 ]
 [-1469.63989668]
 [-1179.6835687 ]]
[[ -764.3018586 ]
 [-1471.59535497]
 [-1179.6835687 ]]
[[ -764.3018586 ]
 [-1471.59535497]
 [-1181.2531675 ]]
[[ -765.31688473]
 [-1471.59535497]
 [-1181.2531675 ]]
[[ -765.31688473]
 [-1473.55081326]
 [-1181.25

...

[[-4089.52745387]
 [-7875.72124861]
 [-6321.68923547]]
[[-4089.52745387]
 [-7877.6767069 ]
 [-6321.68923547]]
[[-4089.52745387]
 [-7877.6767069 ]
 [-6323.25883427]]
[[-4090.54248   ]
 [-7877.6767069 ]
 [-6323.25883427]]
[[-4090.54248   ]
 [-7879.63216518]
 [-6323.25883427]]
[[-4090.54248   ]
 [-7879.63216518]
 [-6324.82843307]]
[[-4091.55750613]
 [-7879.63216518]
 [-6324.82843307]]
[[-4091.55750613]
 [-7881.58762347]
 [-6324.82843307]]
[[-4091.55750613]
 [-7881.58762347]
 [-6326.39803187]]
[[-4092.57253226]
 [-7881.58762347]
 [-6326.39803187]]
[[-4092.57253226]
 [-7883.54308176]
 [-6326.39803187]]
[[-4092.57253226]
 [-7883.54308176]
 [-6327.96763067]]
[[-4093.58755839]
 [-7883.54308176]
 [-6327.96763067]]
[[-4093.58755839]
 [-7885.49854005]
 [-6327.96763067]]
[[-4093.58755839]
 [-7885.49854005]
 [-6329.53722947]]
[[-4094.60258451]
 [-7885.49854005]
 [-6329.53722947]]
[[-4094.60258451]
 [-7887.45399834]
 [-6329.53722947]]
[[-4094.60258451]
 [-7887.45399834]
 [-6331.10682826]]
[[-4095.61

 [-6599.50822296]]
[[-4269.18707853]
 [-8223.7928239 ]
 [-6599.50822296]]
[[-4269.18707853]
 [-8223.7928239 ]
 [-6601.07782176]]
[[-4270.20210465]
 [-8223.7928239 ]
 [-6601.07782176]]
[[-4270.20210465]
 [-8225.74828218]
 [-6601.07782176]]
[[-4270.20210465]
 [-8225.74828218]
 [-6602.64742056]]
[[-4271.21713078]
 [-8225.74828218]
 [-6602.64742056]]
[[-4271.21713078]
 [-8227.70374047]
 [-6602.64742056]]
[[-4271.21713078]
 [-8227.70374047]
 [-6604.21701936]]
[[-4272.23215691]
 [-8227.70374047]
 [-6604.21701936]]
[[-4272.23215691]
 [-8229.65919876]
 [-6604.21701936]]
[[-4272.23215691]
 [-8229.65919876]
 [-6605.78661816]]
[[-4273.24718304]
 [-8229.65919876]
 [-6605.78661816]]
[[-4273.24718304]
 [-8231.61465705]
 [-6605.78661816]]
[[-4273.24718304]
 [-8231.61465705]
 [-6607.35621696]]
[[-4274.26220917]
 [-8231.61465705]
 [-6607.35621696]]
[[-4274.26220917]
 [-8233.57011534]
 [-6607.35621696]]
[[-4274.26220917]
 [-8233.57011534]
 [-6608.92581575]]
[[-4275.27723529]
 [-8233.57011534]
 [-6608.92

 [-6891.45359964]]
[[-4457.98193833]
 [-8585.5526072 ]
 [-6891.45359964]]
[[-4457.98193833]
 [-8587.50806549]
 [-6891.45359964]]
[[-4457.98193833]
 [-8587.50806549]
 [-6893.02319844]]
[[-4458.99696446]
 [-8587.50806549]
 [-6893.02319844]]
[[-4458.99696446]
 [-8589.46352378]
 [-6893.02319844]]
[[-4458.99696446]
 [-8589.46352378]
 [-6894.59279724]]
[[-4460.01199059]
 [-8589.46352378]
 [-6894.59279724]]
[[-4460.01199059]
 [-8591.41898207]
 [-6894.59279724]]
[[-4460.01199059]
 [-8591.41898207]
 [-6896.16239604]]
[[-4461.02701672]
 [-8591.41898207]
 [-6896.16239604]]
[[-4461.02701672]
 [-8593.37444035]
 [-6896.16239604]]
[[-4461.02701672]
 [-8593.37444035]
 [-6897.73199484]]
[[-4462.04204284]
 [-8593.37444035]
 [-6897.73199484]]
[[-4462.04204284]
 [-8595.32989864]
 [-6897.73199484]]
[[-4462.04204284]
 [-8595.32989864]
 [-6899.30159364]]
[[-4463.05706897]
 [-8595.32989864]
 [-6899.30159364]]
[[-4463.05706897]
 [-8597.28535693]
 [-6899.30159364]]
[[-4463.05706897]
 [-8597.28535693]
 [-6900.87

 [-7192.81656912]]
[[-4652.8669549 ]
 [-8961.00059852]
 [-7192.81656912]]
[[-4652.8669549 ]
 [-8962.95605681]
 [-7192.81656912]]
[[-4652.8669549 ]
 [-8962.95605681]
 [-7194.38616792]]
[[-4653.88198103]
 [-8962.95605681]
 [-7194.38616792]]
[[-4653.88198103]
 [-8964.9115151 ]
 [-7194.38616792]]
[[-4653.88198103]
 [-8964.9115151 ]
 [-7195.95576672]]
[[-4654.89700716]
 [-8964.9115151 ]
 [-7195.95576672]]
[[-4654.89700716]
 [-8966.86697339]
 [-7195.95576672]]
[[-4654.89700716]
 [-8966.86697339]
 [-7197.52536552]]
[[-4655.91203329]
 [-8966.86697339]
 [-7197.52536552]]
[[-4655.91203329]
 [-8968.82243168]
 [-7197.52536552]]
[[-4655.91203329]
 [-8968.82243168]
 [-7199.09496432]]
[[-4656.92705942]
 [-8968.82243168]
 [-7199.09496432]]
[[-4656.92705942]
 [-8970.77788996]
 [-7199.09496432]]
[[-4656.92705942]
 [-8970.77788996]
 [-7200.66456312]]
[[-4657.94208554]
 [-8970.77788996]
 [-7200.66456312]]
[[-4657.94208554]
 [-8972.73334825]
 [-7200.66456312]]
[[-4657.94208554]
 [-8972.73334825]
 [-7202.23

KeyboardInterrupt: 

In [32]:
x_coord, i_coord

(matrix([[ 5.54689096],
         [-4.11457168],
         [-1.93871252]]), 176)

In [24]:
f(x_coord)

-2.378133614071034

### Gaussian elimination

Well it is a well known algorithm in linear algebra for soloving systems of linear equations (<a href="https://en.wikipedia.org/wiki/Gaussian_elimination">link</a>)

In [25]:
def gaus(A, b):
    n = A.shape[0]
    for i in range(n):
        b[i] = b[i] / A[i][i]
        A[i] = A[i] / A[i][i]
        for j in range(i + 1, n):
            b[j] = b[j] - A[j][i] * b[i]
            A[j] = A[j] - A[j][i] * A[i]

    for i in range(n - 1, -1, -1):
        for j in range(i - 1, -1, -1):
            b[j] -= b[i] * A[j][i]
    return b

In [26]:
x_pr = gaus(A.copy(), -b.copy())

In [27]:
x_pr

array([[ 5.54690025],
       [-4.11457492],
       [-1.93871834]])

In [18]:
f(x_pr)

-6.574604400639203

### Comparison of methods

#### gradient descent

In [28]:
print("Discrepancy with the exact answer: ", np.linalg.norm(x_sd - x_pr))
print("number of steps: ", i_sd)

Discrepancy with the exact answer:  1.8084252710174648e-06
number of steps:  57


#### coordinate descent 

In [29]:
print("Discrepancy with the exact answer: ", np.linalg.norm(x_coord - x_pr))
print("number of steps: ", i_coord)

Discrepancy with the exact answer:  1.143296702358046e-05
number of steps:  176
